# 2D Projective Geometric Algebra

In [3]:
import clifford as cl
from time import sleep
import numpy as np
from IPython.display import Markdown, Latex
from ipycanvas import hold_canvas
from warnings import filterwarnings
from itertools import permutations
import re
from numba.core.errors import NumbaDeprecationWarning
filterwarnings("ignore", category=NumbaDeprecationWarning)
from anutils.mnp.pga2d.draw_context import PGADrawContext
from anutils.mnp.pga2d import build_tex_format_for_layout
cl.print_precision(2)

# Using 2D Algebra
layout, blades = cl.Cl(2, 0, 1, names=['', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'])
pga_context = lambda: PGADrawContext(layout, blades)
pga_debug_context = lambda: PGADrawContext(layout, blades, debug=True)
e0 = e1 = e2 = e01 = e02 = e12 = e012 = None
globals().update({ k:v for k,v in blades.items() if k != '' })
e20 = -e02
O = e12
I = e012

## PGA Elements


### Vectors are lines

We represent a line as a linear equation (equal to zero) with three coefficients. 

$$
    a\ x + b\ y + c = 0
$$

To represent these lines as vectors, each of these coefficients have their own PGA basis vector $\textbf{e}_i$ in a linear combination.

$$ 
    \textbf{v} = a\ \textbf{e}_1 + b\ \textbf{e}_2 + c\ \textbf{e}_0
$$

In [5]:
v = -3/5*e1 + 2/5*e2 + 100*e0
print(v)

pga_context().draw(v).canvas

(100.0^e0) - (0.6^e1) + (0.4^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Bivectors are points

$$
    \textbf{P} = (x, y) \rightarrow x\ \textbf{e}_{20} + y\ \textbf{e}_{01} + \textbf{e}_{12}
$$

Note: the coefficient $\textbf{e}_{12}$ is usually $1.0$ which represents a normalized bivector. However, in PGA, all scalar multiples of multivectors ultimately represent the same object.

$$
    \textbf{P}_a = x\textbf{e}_{20} + y\textbf{e}_{01} + \textbf{e}_{12}
$$

$$
    \textbf{P}_b = 2x\textbf{e}_{20} + 2y\textbf{e}_{01} + 2\textbf{e}_{12} = P_a
$$

$\textbf{e}_{12}$ can geometrically represent the origin $\textbf{O}$, as this would equate to $(0, 0)$ given the formula above.

In fact, the above formula can be written much more succinctly utilizing the unit pseudoscalar $\textbf{I} = \textbf{e}_{012}$

$$
    \textbf{P} = \textbf{O} + (x\textbf{e}_{1} + y\textbf{e}_{2})\textbf{I}
$$

We can finally, in 2D PGA, represent points as the intersection of offsets to the $x$ and $y$ axes.

$$
    \textbf{P} = (\textbf{e}_1 - x\textbf{e}_0)(\textbf{e}_2 - y\textbf{e}_0)
$$

In [6]:
p = O + I*(-90*e1 + 50*e2) # -90*e20 + 50*e01 + e12
print('p:', p)
print('origin:', O)

pga_context().draw(p).draw(O).canvas

p: (50^e01) + (90^e02) + (1^e12)
origin: (1^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

## PGA Operations

### Wedge Product: Intersections

The wedge product $v \wedge u$ is computed from the above basis vector rules:

$$
    \textbf{v} \wedge \textbf{u} = (v_1 \textbf{e}_1 + v_2 \textbf{e}_2 + v_0 \textbf{e}_0) \wedge (u_1 \textbf{e}_1 + u_2 \textbf{e}_2 + u_0 \textbf{e}_0)
$$

$$
\begin{aligned}
    = && v_1 u_1 (\textbf{e}_1 \wedge \textbf{e}_1) 
    & + & v_1 u_2 (\textbf{e}_1 \wedge \textbf{e}_2) 
    & + & v_1 u_0  (\textbf{e}_1 \wedge \textbf{e}_0) \\
     && v_2 u_1 (\textbf{e}_2 \wedge \textbf{e}_1) 
     & + & v_2 u_2 (\textbf{e}_2 \wedge \textbf{e}_2) 
     & + & v_2 u_0 (\textbf{e}_2 \wedge \textbf{e}_0) \\
     && v_0 u_1 (\textbf{e}_0 \wedge \textbf{e}_1) 
     & + & v_0 u_2 (\textbf{e}_0 \wedge \textbf{e}_2) 
     & + & v_0 u_0 (\textbf{e}_0 \wedge \textbf{e}_0)
\end{aligned}
$$

$$
= (v_1 u_2 - v_2 u_1) \textbf{e}_{12} + (u_1 v_0 - v_1 u_0) \textbf{e}_{01} + (v_2 u_0 - u_2 v_0) \textbf{e}_{20}
$$

Geometrically, treating $v$ and $u$ as lines, this is the intersection of both lines!

In [7]:
v = 1.2*e1 - 0.1*e2 - 100*e0
u = 0.1*e1 - 0.9*e2 +  40*e0
p = v ^ u
print('v:', v)
print('u:', u)
print('v ^ u:', p)
print('normalized:', p/p[e12])

pga_context().draw(v, u, p).canvas

v: -(100.0^e0) + (1.2^e1) - (0.1^e2)
u: (40.0^e0) + (0.1^e1) - (0.9^e2)
v ^ u: -(58.0^e01) + (94.0^e02) - (1.07^e12)
normalized: (54.21^e01) - (87.85^e02) + (1.0^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Intersection of Two Parallel Lines: Point at Infinity

What if the two lines are parallel? By definition they wouldn't have an intersection then. In PGA, this discrepency is handled by describing parallel lines as intersecting at "infinity". Given two parallel lines $\textbf{v}$ and $\textbf{v} + \alpha \textbf{e}_0$, their intersection can be computed using the basis vector rules:

$$
    \textbf{v} \wedge (\textbf{v} + \alpha \textbf{e}_0) = (v_1 \textbf{e}_1 + v_2 \textbf{e}_2 + v_0 \textbf{e}_0) \wedge (v_1 \textbf{e}_1 + v_2 \textbf{e}_2 + (v_0 + \alpha) \textbf{e}_0)
$$

$$
\begin{aligned}
    = && v_1^2   & (\textbf{e}_1 \wedge \textbf{e}_1) + v_1 v_2 & (\textbf{e}_1 \wedge \textbf{e}_2) + v_1 (v_0 + \alpha) & (\textbf{e}_1 \wedge \textbf{e}_0) \\
     && v_2 v_1 & (\textbf{e}_2 \wedge \textbf{e}_1) + v_2^2   & (\textbf{e}_2 \wedge \textbf{e}_2) + v_2 (v_0 + \alpha) & (\textbf{e}_2 \wedge \textbf{e}_0) \\
     && v_0 v_1 & (\textbf{e}_0 \wedge \textbf{e}_1) + v_0 v_2 & (\textbf{e}_0 \wedge \textbf{e}_2) + v_0 (v_0 + \alpha) & (\textbf{e}_0 \wedge \textbf{e}_0)
\end{aligned}
$$

$$
    = (v_1 v_2 - v_2 v_1) \textbf{e}_{12} + (v_1 (v_0 + \alpha) - v_0 v_1) \textbf{e}_{10} + (v_2 (v_0 + \alpha) - v_0 v_2) \textbf{e}_{20}
$$

$$
    = (\alpha v_1) \textbf{e}_{10} + (\alpha v_2) \textbf{e}_{20}
$$

$$
    = \textbf{I} ( - v_2 \textbf{e}_1 + v_1 \textbf{e}_2 )
$$

This appears to be a point without an origin value $\textbf{O} = \textbf{e}_{12}$. This would be considered a "Point at Infinity". To visualize this, we can add an $\textbf{O}$ back in and vary the scale of the origin point to see what happens to the point. As the scale of $\textbf{O}$ goes to $0$, the point travels in the direction along both parallel lines away from the screen, into what we would call "Infinity".

In [8]:
v = -4/5 * e1 + 1/5 * e2 + 50 * e0
a = 100

# Intersection between parallel lines
d = v ^ (v + a * e0)

# The trend towards infinity as O gets closer to 0
ps = np.linspace(1, 0, 8)*O + d

pga_context().draw(v, (v + a*e0), *ps).canvas

Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Join two points: Vee Product

Find the line passing through two points using the regressive (or vee) product: $\vee$

In [9]:
p1 = O + I*(-40*e1 + 90*e2)
p2 = O + I*(100*e1 - 10*e2)
v = p1 & p2
print('p1:', p1)
print('p2:', p2)
print('v:', v)

pga_context().draw(v, p1, p2).canvas

p1: (90^e01) + (40^e02) + (1^e12)
p2: -(10^e01) - (100^e02) + (1^e12)
v: -(8600^e0) + (100^e1) + (140^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Join two points Pt. 2: Computing the distance

Since scalar multiples of PGA objects don't affect the geometry of that object, they offer a gauge symmetry that can actually be useful in encoding information within that objects. This gives the join operator another trick up it's sleeve. The operation not only produces the line between two points, the resulting line is scaled by the distance between the two points! To see this, we should first remind ourselves of the classical distance given two points: $P_a = (x_a,\ y_a)$ and $P_b = (x_b,\ y_b)$. We will focus on the squared distance for now since finding the actual distance after that is trivial.

$$
    \text{dist}\left(P_b,\ P_a\right)^2 = |x_b - x_a|^2 + |y_b - y_a|^2
$$

Now let's convert to PGA use the join operator and focus it's effects on the basis vectors. 

$$
\begin{align*}
     P_a &\rightarrow \textbf{P}_a &= x_a \textbf{e}_{20} + y_a \textbf{e}_{01} + \textbf{e}_{12} \\
     P_b &\rightarrow \textbf{P}_b &= x_b \textbf{e}_{20} + y_b \textbf{e}_{01} + \textbf{e}_{12} 
\end{align*}
$$


The join operator $\vee$ can actually be computed using the meet $\wedge$ of the duals of the basis multivectors. The dual (in PGA) of a basis multivector $\textbf{X}$ is another multivector $\star\textbf{X}$ which, in a geometric product with $\textbf{X}$ produces the unit pseudoscalar $\textbf{I} = \textbf{e}_{012}$. For example: the dual of $\textbf{e}_{20}$ labeled ($\star \textbf{e}_{20}$) will be $\textbf{e}_1$, since $\textbf{e}_{20} \textbf{e}_1 = \textbf{e}_{201} = \textbf{e}_{012} = \textbf{I}$. Given that, the join operation $\textbf{X} \vee \textbf{Y}$ can be expressed as $\star\left(\star\textbf{X} \wedge \star\textbf{Y}\right)$. It's important to note that the join (along with the meet, the dot product, and the geometric product for that matter) are all bilinear operations, meaning that the join of two linear combinations of basis vectors can be computed if you know the joins of the individual pairs of basis vectors.

$$
\begin{align*}
    \textbf{P}_a \vee \textbf{P}_b &= \star\left(\star\textbf{P}_a \wedge \star\textbf{P}_b\right) \\
    &= \star\left(\star \left(x_a \textbf{e}_{20} + y_a \textbf{e}_{01} + \textbf{e}_{12}\right) \wedge \star \left(x_b \textbf{e}_{20} + y_b \textbf{e}_{01} + \textbf{e}_{12}\right) \right) \\
    &= \star \left( \left(x_a \textbf{e}_{1} + y_a \textbf{e}_{2} + \textbf{e}_{0}\right) \wedge \left(x_b \textbf{e}_{1} + y_b \textbf{e}_{2} + \textbf{e}_{0}\right) \right)
\end{align*}
$$

Compute the wedge products between the duals:

$$
\begin{align*}
      \left(x_a \textbf{e}_{1} + y_a \textbf{e}_{2} + \textbf{e}_{0}\right) \wedge \left(x_b \textbf{e}_{1} + y_b \textbf{e}_{2} + \textbf{e}_{0}\right)
    &= x_a x_b \left( \textbf{e}_{1} \wedge \textbf{e}_{1} \right) 
    + x_a y_b \left( \textbf{e}_{1} \wedge \textbf{e}_{2} \right) 
    + x_a \left( \textbf{e}_{1} \wedge \textbf{e}_{0} \right) \\   
    &+ y_a x_b \left( \textbf{e}_{2} \wedge \textbf{e}_{1} \right) 
    + y_a y_b \left( \textbf{e}_{2} \wedge \textbf{e}_{2} \right) 
    +     y_a \left( \textbf{e}_{2} \wedge \textbf{e}_{0} \right) \\   
    &+     x_b \left( \textbf{e}_{0} \wedge \textbf{e}_{1} \right) 
    +     y_b \left( \textbf{e}_{0} \wedge \textbf{e}_{2} \right) 
    +         \left( \textbf{e}_{0} \wedge \textbf{e}_{0} \right) \\ 
    &= \left( x_a y_b - x_b y_a \right) \textbf{e}_{12}
     + \left( y_a - y_b \right) \textbf{e}_{20}
     + \left( x_b - x_a \right) \textbf{e}_{01}
\end{align*}
$$

Now take the dual of the result

$$
\begin{align*}
\star \left( \left( x_a y_b - x_b y_a \right) \textbf{e}_{12}
     + \left( y_a - y_b \right) \textbf{e}_{20}
     + \left( x_b - x_a \right) \textbf{e}_{01} \right) \\
= \left( x_a y_b - x_b y_a \right) \textbf{e}_{0}
     + \left( y_a - y_b \right) \textbf{e}_{1}
     + \left( x_b - x_a \right) \textbf{e}_{2}
\end{align*}
$$

If we take the geometric product of this line with itself, the result is the magnitude squared of the vector.

$$
\left( \left( x_a y_b - x_b y_a \right) \textbf{e}_{0}
     + \left( y_a - y_b \right) \textbf{e}_{1}
     + \left( x_b - x_a \right) \textbf{e}_{2} \right) ^ 2 \\
= \left( y_a - y_b \right)^2 + \left( x_b - x_a \right)^2 \\
= |x_b - x_a|^2 + |y_b - y_a|^2
$$

So as a result, if we want to compute the distance between two points, we join the line between them and compute the magnitude of the line!

$$
     \text{dist}(P_b,\ P_a)^2 = ||\textbf{P}_b \vee \textbf{P}_a||^2
$$

In [10]:
p1 = O + I*(-100*e1 + 90*e2)
p2 = O + I*(100*e1 - 10*e2)
p3 = O + I*(190*e1 + 120*e2)
v12 = p1 & p2
v13 = p1 & p3
v23 = p2 & p3
print('p1:', p1)
print('p2:', p2)
print('p3:', p3)
print('v1:', v12)
print('v2:', v13)
print('v3:', v23)
print('dist(p1, p2)^2:', v12.mag2())
print('dist(p1, p3)^2:', v13.mag2())
print('dist(p2, p3)^2:', v23.mag2())

pga_context().draw(v12, v13, v23, p1, p2, p3).canvas

p1: (90^e01) + (100^e02) + (1^e12)
p2: -(10^e01) - (100^e02) + (1^e12)
p3: (120^e01) - (190^e02) + (1^e12)
v1: -(8000^e0) + (100^e1) + (200^e2)
v2: -(29100^e0) - (30^e1) + (290^e2)
v3: (13900^e0) - (130^e1) + (90^e2)
dist(p1, p2)^2: 50000
dist(p1, p3)^2: 85000
dist(p2, p3)^2: 25000


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Perpendicular lines: Dot Product

Normally the dot product between two vectors returns a scalar value indicating the alignment and magnitude between lines. However, the dot product between a vector $\textbf{v}$ and a bivector $\textbf{P}$ returns a vector which will actually be the line perpendicular to $\textbf{v}$ passing through $\textbf{P}$

$$
    \textbf{v} \cdot \textbf{P}
$$

In [11]:
v = - 1*e1 + 2*e2 + 270*e0
p = O + I*(40*e1 + 5*e2)
vp = v | p
print('v:',   v)
print('p:',   p)
print('vp:', vp)

pga_context().draw(v, vp, p).canvas

v: (270^e0) - (1^e1) + (2^e2)
p: (5^e01) - (40^e02) + (1^e12)
vp: (85^e0) - (2^e1) - (1^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Projections

Project a point $\textbf{P}$ onto a line $\textbf{v}$ using the formula: 

$$
    \frac{\textbf{v} \cdot \textbf{P}}{\textbf{v}}
$$

In [12]:
v = - 1*e1 + 2*e2 + 270*e0
p = O + I*(40*e1 + 5*e2)
ponv = (v | p) / v
print('v:', v)
print('p:', p)
print('p on v:', ponv)

pga_context().draw(v, p, ponv).canvas

v: (270^e0) - (1^e1) + (2^e2)
p: (5^e01) - (40^e02) + (1^e12)
p on v: (91.0^e01) + (88.0^e02) - (1.0^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

You can also use the same formula to project a line $\textbf{v}$ onto a point $\textbf{P}$ using the following:

$$
\frac{\textbf{v} \cdot \textbf{P}}{\textbf{P}}
$$

In [13]:
v = - 1*e1 + 2*e2 + 270*e0
p = O + I*(40*e1 + 5*e2)
vonp = (v | p) / p
print('v:', v)
print('p:', p)
print('v on p:', vonp)

pga_context().draw(v, vonp, p).canvas

v: (270^e0) - (1^e1) + (2^e2)
p: (5^e01) - (40^e02) + (1^e12)
v on p: (30.0^e0) - (1.0^e1) + (2.0^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Midpoints

Find the midpoint between two points: just add them!

In [14]:
p1 = O + I*( -40*e1 + 90*e2 )
p2 = O + I*( 100*e1 - 10*e2 )
pM = p1 + p2
print('p1:', p1)
print('p2:', p2)
print('pM:', pM, 'normalized:', pM/pM[e12])

pga_context().draw(p1, p2, pM).canvas

p1: (90^e01) + (40^e02) + (1^e12)
p2: -(10^e01) - (100^e02) + (1^e12)
pM: (80^e01) - (60^e02) + (2^e12) normalized: (40.0^e01) - (30.0^e02) + (1.0^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

## Transformations of Objects

Now we tap into the true power of PGA.

### Reflection

The simplest transformation of objects in 2D PGA is a reflection. In fact, all transformations can be modeled by composing reflections, as will be seen below. Reflections are modeled entirely with the line of reflection $\textbf{v}$. Any PGA object (be it points or lines), can be reflected over such a line with the following formula:

$$
    P \rightarrow \tilde{\textbf{v}} P \textbf{v}
$$

$\tilde{\textbf{v}}$ in this case is known as the "reverse" operator. Think of it like the quaternion conjugate. It will be more apparent what this means in later cells

In [15]:
obj = cl.array([
    O + I*(20*e1 - 40*e2),
    O + I*(40*e1 + 40*e2),
    O + I*(80*e1 - 30*e2)
])
v = 3*e1 - 1*e2 - 400*e0
ref = ~v * obj * v

ctx = pga_context()
ctx.draw(v)
ctx.draw_polygon(obj, color='green', line_width=2.0)
ctx.draw_polygon(ref, color='orange', line_width=2.0)
ctx.canvas

Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Rotation

If we take two lines $\textbf{v}$ and $\textbf{u}$ which intersect at a point $\textbf{u} \wedge \textbf{v}$, we will do two reflections. The result of this will actually be a rotation around $\textbf{u} \wedge \textbf{v}$. The rotation angle will be twice the angle between lines $\textbf{v}$ and $\textbf{u}$.

$$
    \textbf{P} \rightarrow \tilde{\textbf{v}} \tilde{\textbf{u}} \textbf{P} \textbf{u} \textbf{v}
$$

We can do the reflections independently as above, or we can combine the lines $\textbf{u}$ and $\textbf{v}$ with the geometric product $\textbf{u}\textbf{v}$. This produces an object $\textbf{m}$ called a _motor_. 

$$
    \textbf{m} = \textbf{u}\textbf{v}
$$

It's important to note that this is different from simply the point $\textbf{u} \wedge \textbf{v}$, as it contains more information regarding the rotation action.

The reverse operator $\tilde{\textbf{m}}$ can actually be understood algebraically as taking the product $\textbf{u}\textbf{v}$ and reversing the order of the terms in the product (while also computing the reverses of the terms)

$$
    \tilde{\textbf{m}} = \tilde{\textbf{v}}\tilde{\textbf{u}}
$$

The above operation can be simplified to just using the motor

$$
    \textbf{P} \rightarrow \tilde{\textbf{m}} \textbf{P} \textbf{m}
$$

In [16]:
obj = cl.array([
    O + I*(20*e1 - 40*e2),
    O + I*(40*e1 + 40*e2),
    O + I*(80*e1 - 30*e2)
])

v = -2*e1 + 1*e2 + 500*e0
u = -3*e1 - 1*e2 + 300*e0
m = u*v
p = u ^ v
print('Point P:', p)
print('Motor m:', m)

ref = ~u*obj*u
rot = ~m*obj*m

ctx = pga_context()
ctx.draw(u, v, p)
ctx.draw_polygon(obj, color='green', line_width=2.0)
ctx.draw_polygon(ref, color='#ccc', line_width=1.0)
ctx.draw_polygon(rot, color='orange', line_width=2.0)
ctx.canvas

Point P: (900^e01) + (800^e02) - (5^e12)
Motor m: 5 + (900^e01) + (800^e02) - (5^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Translations

Parallel lines intersect at infinity. If we take the geometric product of two parallel lines $\textbf{v}$ and $\textbf{u} = \textbf{v} + \alpha \textbf{e}_0$, the resulting motor will be a rotation around a point at infinity. In other words: a translation

In PGA, all motion (translation and rotation) is modeled as a rotation around a point, either on screen or at infinity.

$$
    \textbf{m} = \textbf{v} (\textbf{v} + \alpha \textbf{e}_0)
$$

$$
    \textbf{P} \rightarrow \tilde{\textbf{m}} \textbf{P} \textbf{m}
$$

In [17]:
obj = cl.array([
    O + I*(20*e1 - 40*e2),
    O + I*(40*e1 + 40*e2),
    O + I*(80*e1 - 30*e2)
])

u = 2/3*e1 - 1/3*e2 - 70*e0
v = u - 70*e0
m = u*v
p = u ^ v
print('Point P:', p)
print('Motor m:', m)

ref = ~u*obj*u
trs = ~m*obj*m

ctx = pga_context()
ctx.draw(u, v, p)
ctx.draw_polygon(obj, color='green', line_width=2.0)
ctx.draw_polygon(ref, color='#ccc', line_width=1.0)
ctx.draw_polygon(trs, color='orange', line_width=2.0)
ctx.canvas

Point P: (46.67^e01) - (23.33^e02)
Motor m: 0.56 + (46.67^e01) - (23.33^e02)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### General Motion

A generic motor can be made using the exponential operation. Take any bivector $\textbf{P}$ (either a regular point or a point at infinity, considered the axis of rotation/translation) and scale it by a factor proportional to half the angle of rotation (or distance of translation) to be performed and raise $e$ to the result

$$
    \textbf{m} = e^{\frac{\theta}{2} \textbf{P}}
$$

The benefit of this is that combinations of rotations and translations can be modeled as a single rotation around an offset axis

In [18]:
obj = cl.array([
    O + I*(20*e1 - 40*e2),
    O + I*(40*e1 + 40*e2),
    O + I*(80*e1 - 30*e2)
])

p = O + I*(100*e1 + 200*e2)
angle = 45 * np.pi/180

m = (angle/2 * p).exp()
mot = ~m*obj*m

ctx = pga_context()
ctx.draw(p)
ctx.draw_polygon(obj, color='green', line_width=2.0)
ctx.draw_polygon(mot, color='orange', line_width=2.0)
ctx.canvas

Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

## Rendering

PGA is powerful enough that many rendering tasks can be achieved through PGA operations. In fact, the above demos have been drawn largely using PGA!

In [19]:
pga_debug_context().canvas

Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

### Drawing Lines

For an infinite line, given that the two ends of the line are known, we can use common algorithms like bresnaham's line algorithm to draw the line between the ends. 

Now for finding the ends of a line, what if we considered the bounds of the screen to themselves be PGA lines. Then we can simply compute the intersection of our line with each of the boundaries to get the points needed to draw the full line.

This will however find four points. We will need the two points within the screen, so we can use the distance formula described above to find the two closest points relative to the center. These will be the boundaries of our lines

In [20]:
width = 600
height = 400

bounds = cl.array([
     e1 - width//2*e0,
    -e1 - width//2*e0,
     e2 - height//2*e0,
    -e2 - height//2*e0
])
v = 1.125*e1 - 2*e2 + 200*e0
intersects = v ^ bounds

ctx = pga_context()
for b in bounds:
    ctx.draw_line(b, color='#ccc')
ctx.draw_line(v, color='#9999ff')
for i in intersects:
    if i[e12] != 0:
        ctx.draw_circle(i, color='#ff99ff', radius=5)
a, b = sorted(intersects, key=lambda a: (e12 & a.normal()).mag2())[:2]
q = ctx._reference_motor()
ctx._draw_line_segment(~q*a*q, ~q*b*q, color='#0000ff')
ctx.draw_circle(a, color='#ff00ff', radius=7)
ctx.draw_circle(b, color='#ff00ff', radius=7)
ctx.draw_circle(e12, color='#999', radius=5)
ctx.canvas

Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)